# Simple NN

## Build the basci NN to clasify IRIS flower

- We have input layer, output laeyer and hidden layers. 
- Each layer has different neurons.
- Each neuron is connected to other neurons in other layer. It s called fully connected layer.
- So we are doing some calcualtion and movinf __forward__ after each layer.
- Finally it mocve to output layer

Task: create a model to classify IRIS flowers. It has three class Iris Setosa, Iris Versicolor, Iris Virginica

Dataset: IRIS dataset https://archive.ics.uci.edu/dataset/53/iris

| S.No           | Variable Name  | Role  | Type       | Description | Units | Missing Values |
|--------------- |---------------|--------|------------|-------------|-------|---------------|
|  F1   | sepal length  | Feature | Continuous |             | cm    | no            |
|  F2   | sepal width   | Feature | Continuous |             | cm    | no            |
|  F3   | petal length  | Feature | Continuous |             | cm    | no            |
|  F4   | petal width   | Feature | Continuous |             | cm    | no            |
|  C   | class         | Target  | Categorical |  class of iris plant: Iris Setosa, Iris Versicolour, or Iris Virginica |    |  no   |

Create a NN model with two hidden layers (h1,h2).  IRIS dataset has 4 features so our input layer will have size 4. Output will be Iris flower class whcih is 3 (Iris Setosa, Iris Versicolour, or Iris Virginica)

<center><img src='./images/iris_NN.PNG' width=600px></center> 

Ref: https://playground.tensorflow.org/#activation=tanh&batchSize=10&dataset=circle&regDataset=reg-plane&learningRate=0.03&regularizationRate=0&noise=0&networkShape=4,2&seed=0.89586&showTestData=false&discretize=false&percTrainData=50&x=true&y=true&xTimesY=false&xSquared=false&ySquared=false&cosX=false&sinX=false&cosY=false&sinY=false&collectStats=false&problem=classification&initZero=false&hideText=false

# How to create NN in PyToerch. 
Steps

1. 
2. 






## 1. Create model

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F # help to move data forward

import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd

In [ ]:
# Create a model class that inherits `nn.Module`
# We will use calss based OOP in PyTorch. We will create a class and instantiate the class

# Create a NN model, which has input layer (4 features), two hidden layer with neurons and Output layer with 3 class.
class Model(nn.Module): # creating a class whcih inherites `nn.Module`
    # Create each layer of the NN model that you want to build. Add those in the model individualy
    # Input Layer (4 features of the flower) --> 
    # Hidden Layer1 (# of neurons) --> H2 (n) --> 
    # output (3 classes of IRIS flower)

    # define a method or function for to create NN layers.
    def __init__(self, in_feateures=4, h1=8, h2=9, out_features=3):
        super().__init__() # instantiate our nn.Module
        # setup fully connected hidden layers 
        self.fc1 = nn.Linear(in_feateures, h1)
        self.fc2 = nn.Linear(h1, h2)
        self.out =nn.Linear(h2,out_features)

    # Define a function whcih moves everython forward
    # This will be used in training and testing the data.
    def forward(self, x):  # x is the input data
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.out(x)

        return x
    

In [ ]:
# intiialise with seed for randomization
torch.manual_seed(41)

# Create an instance of model and call it
model = Model()

## 2. Load Data: train, test Split

In [ ]:
# Load Data and Train/Test Neural Network Model
url = 'https://gist.githubusercontent.com/curran/a08a1080b88344b0c8a7/raw/0e7a9b0a5d22642a06d3d5b9bcbad9890c8ee534/iris.csv'
# create adataframe
my_df = pd.read_csv(url)

my_df
my_df.head()
my_df.tail()

In [ ]:
# Data preprocessing.
# We change the class / variety the last column of data in to numbers instead of name (string).
my_df['variety'] = my_df['variety'].replace('Setosa', 0.0)
my_df['variety'] = my_df['variety'].replace('Versicolor', 1.0)
my_df['variety'] = my_df['variety'].replace('Virginica', 2.0)

my_df.head()

In [ ]:
# Arrange Train and test data. Set X: feature, y: output (class)

X = my_df.drop('variety', axis=1) # delete the last column and just keep features for matrix X
y = my_df['variety'] # keep last column for y (output/ class) or labels

# Convert dataframe to numpy array
X = X.values
y = y.values

In [ ]:
# Do train, test , split using sklearn. Traini size = 80%, test size = 20%
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=41)

# convert X feateures in to tensor
X_train = torch.FloatTesnor(X_train)
X_test = torch.FloatTesnor(X_test)

# convert y labels in to tensors long tensor, i.e 64 bit integers.
y_train = torch.LongTesnor(y_train)
y_test = torch.LongTesnor(y_test)

##  3. Define the Optimization routine for running NN in loop.

In [ ]:
# Loss and optimizer
# Set the criterion odf model to measure the error, how far off the prediction are from the data
criterion = nn.CrossEntropyLoss()

# Choose Adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.01) # model parameters are different layers . 
# PyTorch will automatically grab them from the model

# print model parametres
model.parameters

In [ ]:
# train the model
# Epochs? (one run thru all training data in our network)

epochs = 100
losses = [] # keep track of loss

for i in range(epochs):
    # Go forward and get prediction
    y_pred = model.forward(X_train) # Get predicted  results

    # Measure the loss/error, gonna be hight at first
    loss = criterion(y_pred, y_train) # predicted value vs the y_train

    # Keep track of losses
    losses.append(loss.detach().numpy())

    # print every 10 epoch
    if i % 10 == 0:
        print(f'Epoch: {i} and loss: {loss}')

    # Do backpropagation: take the error rate of forwrad propagation and feed it back thru the netowrk to fine tune the weights
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

plt.plot(epocs, losses)
plt.xlabel('Epoch')
plt.ylabel('Loss/Error')

## 4. Evaluate model on Test Data Set On Network
Validate model on test set

In [ ]:

with torch.no_grad():  # Basically turn off back-propagation
    # this time for testing model we are giving X_test data not X_train. 
    # X_test are the featers from our test set, y_eval will be predictions
    y_eval = model.forward(X_test) 
    loss = criterion(y_eval, y_test) # find loss or error


loss # this loss is not acceptable. Above was trainign loss and this is testing loss

In [ ]:
correct =0 
with torch.no_grad():
    for i, data in enumerate(X_test):
        y_val = model.forward(data) 

        if y_test[i] ==0:
            x = "Setosa"
        elif y_test[i]==1:
            x = 'Versicolor'
        elif y_test[i]==2:
            x = 'Virginica'
        
        # This tell the category of flower
        # print(f'{i+1}.) {str(y_val)} \t {y_test[i]} \t {y_val.argmax().item()}')
        print(f'{i+1}.) {str(y_val)} \t {x} \t {y_val.argmax().item()}')

        # Correct or not
        if y_val.argmax().item() == y_test[i]:
            correct += 1

print(f'We got {correct} correct!')
        

# We will see three clasees for each row. max probabalilty will give the class

## 5. Evaluate unseen Data On The Network 

In [ ]:
# Create new data point
new_iris = torch.tensor([4.7, 3.2, 1.3, 0.2])

# Feed it back to NN and calssify this flower
with torch.no_grad():
    print(model(new_iris))

## 6. Save and Load our Neural Network Model

In [ ]:
# Save weights and biases about the mdoel into dictionary
torch.save(model.state_dict(), 'IrisNN_model.pt')

In [ ]:
# Load the saved model
new_model = Model() 
new_model.load_state_dict(torch.load('IrisNN_model.pt'))

In [ ]:
# make sure it loaded correctly
new_model.eval()

# Now we can test the data on this new model samee as we did for 'model'